In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import (
    f1_score, 
    precision_score, 
    recall_score, 
    accuracy_score, 
    classification_report
)
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle

In [2]:
data = pd.read_pickle("/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/cubert_embeddings/processed_embeddings/cubert_embed_line_by_line_avg.pkl")
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,-0.673044,-0.426519,-0.730446,0.135270,0.240882,-0.299444,-0.081646,-1.650780,0.836449,0.554071,...,0.223144,-0.566862,-0.050642,0.790394,-0.679357,0.765604,-0.684285,1,12150,critical
1,0.954283,-1.882188,-0.473502,0.119748,-1.110488,-0.443231,0.561635,-0.674690,0.885820,0.760387,...,0.092943,0.524950,-0.121175,0.480204,-0.743649,1.266763,-0.823656,1,10168,critical
2,0.094693,-0.651028,-1.009672,0.473056,0.394086,0.199512,0.299025,-0.518506,0.670121,0.165829,...,0.685719,0.257771,0.018428,0.311697,0.502811,-0.001555,1.064599,1,12783,critical
3,-0.057608,-1.053193,-1.154026,0.371089,-0.191268,-0.746910,0.934388,-0.950130,0.183388,-0.134601,...,-0.309903,-0.422187,-0.038162,0.447833,-0.307583,0.608918,-0.078509,1,12754,critical
4,0.111840,-0.395166,-0.797747,0.429570,0.260171,-0.884858,0.182291,-0.175354,0.413459,0.451911,...,0.276463,0.321459,0.029099,0.301428,-0.008018,0.659282,-0.336579,1,4277,critical


In [3]:
# Separate features and labels
X = data[[col for col in data.columns if col.startswith("emb_")]].values  # Select embedding columns
y = data['label'].values 

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define classifiers
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
bagging = BaggingClassifier(estimator=RandomForestClassifier(), n_estimators=100, random_state=42)  
xgboost = xgb.XGBClassifier(n_estimators=100, random_state=42)

# Add the classifiers to a dictionary
classifiers = {
    "Random Forest": random_forest,
    "Bagging": bagging,
    "XGBoost": xgboost
}

In [4]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to store cross-validation results
results = {}

# Perform cross-validation for each classifier
for clf_name, clf in classifiers.items():
    metrics_summary = {
        "f1_micro": [],
        "f1_macro": [],
        "accuracy": [],
        "precision_macro": [],
        "recall_macro": []
    }
    
    for train_idx, val_idx in skf.split(X_train, y_train):
        # Split the training data into training and validation folds
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
        # Train the classifier
        clf.fit(X_train_fold, y_train_fold)
        
        # Make predictions on the validation set
        y_pred = clf.predict(X_val_fold)
        
        # Calculate metrics
        metrics_summary["f1_micro"].append(f1_score(y_val_fold, y_pred, average='micro'))
        metrics_summary["f1_macro"].append(f1_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["accuracy"].append(accuracy_score(y_val_fold, y_pred))
        metrics_summary["precision_macro"].append(precision_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["recall_macro"].append(recall_score(y_val_fold, y_pred, average='macro'))
    
    # Store mean metrics for the classifier
    results[clf_name] = {metric: np.mean(scores) for metric, scores in metrics_summary.items()}

# Print cross-validation results
print("\nCross-Validation Results:")
for clf_name, metrics in results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")


Cross-Validation Results:

Random Forest Metrics:
F1_micro: 0.8310
F1_macro: 0.5723
Accuracy: 0.8310
Precision_macro: 0.5951
Recall_macro: 0.5644

Bagging Metrics:
F1_micro: 0.8489
F1_macro: 0.5258
Accuracy: 0.8489
Precision_macro: 0.5888
Recall_macro: 0.5279

XGBoost Metrics:
F1_micro: 0.8310
F1_macro: 0.5738
Accuracy: 0.8310
Precision_macro: 0.5954
Recall_macro: 0.5659


In [5]:
# Final evaluation on the test set
final_results = {}

for clf_name, clf in classifiers.items():
    # Train the classifier on the entire training set
    clf.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_test = clf.predict(X_test)
    
    # Calculate test metrics
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    precision_test = precision_score(y_test, y_pred_test, average='macro')
    recall_test = recall_score(y_test, y_pred_test, average='macro')
    accuracy_test = accuracy_score(y_test, y_pred_test)
    report_test = classification_report(y_test, y_pred_test)
    
    # Save final metrics for the classifier
    final_results[clf_name] = {
        "f1_micro": f1_micro_test,
        "f1_macro": f1_macro_test,
        "precision_macro": precision_test,
        "recall_macro": recall_test,
        "accuracy": accuracy_test,
        "classification_report": report_test
    }

# Print final test results
print("\nFinal Test Results:")
for clf_name, metrics in final_results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        if metric != "classification_report":
            print(f"{metric.capitalize()}: {value:.4f}")
    print(f"Classification Report:\n{metrics['classification_report']}")



Final Test Results:

Random Forest Metrics:
F1_micro: 0.8196
F1_macro: 0.5621
Precision_macro: 0.5745
Recall_macro: 0.5562
Accuracy: 0.8196
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.92      0.90       786
           1       0.27      0.20      0.23       123

    accuracy                           0.82       909
   macro avg       0.57      0.56      0.56       909
weighted avg       0.80      0.82      0.81       909


Bagging Metrics:
F1_micro: 0.8471
F1_macro: 0.5417
Precision_macro: 0.5961
Recall_macro: 0.5378
Accuracy: 0.8471
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.96      0.92       786
           1       0.32      0.11      0.17       123

    accuracy                           0.85       909
   macro avg       0.60      0.54      0.54       909
weighted avg       0.80      0.85      0.81       909


XGBoost Metrics:
F1_micro: 0.8207
F1_macro: 

In [7]:
# Save results to a pickle file
output_path = "/Users/mac/Desktop/Code_Smell_Detection/feature_envy/results/cubert/result_cubert_avg.pkl"
with open(output_path, "wb") as f:
    pickle.dump({"cross_val_results": results, "final_results": final_results}, f)
print(f"\nResults saved to {output_path}")



Results saved to /Users/mac/Desktop/Code_Smell_Detection/feature_envy/results/cubert/result_cubert_avg.pkl
